In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


sklearn version:  1.2.2
TF version:  2.10.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


# 다. Baseline 학습 및 결과분석(ResNet50)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import BatchNormalization, GlobalAveragePooling2D, MaxPooling2D, Add,  Dense, Conv2D, Activation
from keras.models import Sequential

In [ ]:
x_train=pd.read_csv('emnist-byclass-train.csv',header=None)
x_test=pd.read_csv('emnist-byclass-test.csv',header=None)

In [ ]:
def data_split(data_type_train,split_ratio=0.1):
    split = StratifiedShuffleSplit(n_splits=1, test_size=split_ratio, random_state=42)
    for train_idx, test_idx in split.split(data_type_train, data_type_train[0]):
        x_train = data_type_train.loc[train_idx]
        x_valid = data_type_train.loc[test_idx]
    return x_train,x_valid

In [ ]:
def data_load(data_type_train,data_type_valid,data_type_test):
    x_train =data_type_train.iloc[:, 1:].values
    y_train = data_type_train.iloc[:, 0].values
    x_valid =data_type_valid.iloc[:, 1:].values
    y_valid = data_type_valid.iloc[:, 0].values
    x_test = data_type_test.iloc[:, 1:].values
    y_test = data_type_test.iloc[:, 0].values
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    return x_train,y_train,x_valid,y_valid,x_test,y_test

In [ ]:
X_train,X_valid=data_split(X_train,0.235)

In [ ]:
X_train,y_train,X_valid,y_valid,X_test,y_test=data_load(X_train,X_valid,X_test)

In [ ]:
np.save('X_train',np.array(X_train,dtype=np.uint8))
np.save('y_train',np.array(y_train,dtype=np.uint8))
np.save('X_valid',np.array(X_valid,dtype=np.uint8))
np.save('y_valid',np.array(y_valid,dtype=np.uint8))
np.save('X_test',np.array(X_test,dtype=np.uint8))
np.save('y_test',np.array(y_test,dtype=np.uint8))

In [ ]:
import csv
def load_Emist():
    X_train=np.load('X_train.npy')
    y_train=np.load('y_train.npy')
    X_valid=np.load('X_valid.npy')
    y_valid=np.load('y_valid.npy')
    X_test=np.load('X_test.npy')
    y_test=np.load('y_test.npy')
    return X_train, y_train, X_valid, y_valid, X_test,y_test


In [ ]:
X_train, y_train, X_valid, y_valid, X_test, y_test = load_Emist()
X_train.shape,X_train.dtype

((533917, 28, 28), dtype('uint8'))

In [ ]:
#data, batch size 성정
train_size=len(X_train)
valid_size=len(X_valid)
batch_size=32

In [ ]:
X_train=np.reshape(X_train,[-1,784])
X_valid=np.reshape(X_valid,[-1,784])

In [ ]:
y_train=np.reshape(y_train,[-1,1])
y_valid=np.reshape(y_valid,[-1,1])

In [ ]:
train_full = np.append(X_train,y_train,axis=1)
valid_full = np.append(X_valid,y_valid,axis=1)

In [ ]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    
    Emnist_dir = os.path.join("datasets", "Emnist")
    os.makedirs(Emnist_dir, exist_ok=True)
    path_format = os.path.join(Emnist_dir, "my_{}_{:02d}.csv")
    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        try:
            with open(part_csv, "xt", encoding="utf-8") as f:
                if header is not None:
                    f.write(header)
                    f.write("\n")
                for row_idx in row_indices:
                    f.write(",".join([repr(col) for col in data[row_idx]]))
                    f.write("\n")
        except:
            continue
    return filepaths

In [ ]:
train_filepaths = save_to_multiple_csv_files(train_full, "train")
valid_filepaths = save_to_multiple_csv_files(valid_full, "valid")

### ResNet50에 맞는 preprocess로 바꿔줌

In [ ]:
n_inputs = X_train.shape[-1]
def preprocess_ResNet50(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    x=tf.reshape(x,[28,28,1])
    y = tf.stack(fields[-1:])
    resized_image = tf.image.resize(x, [224, 224])
    if resized_image.shape[2] == 1:
        temp=tf.concat([resized_image,resized_image],2)
        resized_image=tf.concat([temp,resized_image],2)
    final_image = keras.applications.resnet50.preprocess_input(resized_image)
    
    return final_image, y

In [ ]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess_ResNet50, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [ ]:
train_set = csv_reader_dataset(train_filepaths,batch_size=batch_size, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths,batch_size=batch_size, repeat=None)

# optimizer

### adam

In [ ]:
base_model = tf.keras.applications.ResNet50(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(62, activation="softmax")(avg)
model2 = tf.keras.Model(inputs=base_model.input, outputs=output)
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
early_stopping_BRN50 = EarlyStopping(patience = 10)
data_name="byclass" 
type_name="RN50"
checkpoint_callback_BRN50 = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history2=model2.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping_BRN50, checkpoint_callback_BRN50])

Epoch 1/1000
16684/16684 [==============================] - ETA: 0s - loss: 0.5963 - accuracy: 0.8091
Epoch 1: val_loss improved from inf to 0.51094, saving model to byclass_RN50.h5
16684/16684 [==============================] - 812s 48ms/step - loss: 0.5963 - accuracy: 0.8091 - val_loss: 0.5109 - val_accuracy: 0.8266
Epoch 2/1000
16684/16684 [==============================] - ETA: 0s - loss: 0.4843 - accuracy: 0.8371
Epoch 2: val_loss improved from 0.51094 to 0.48097, saving model to byclass_RN50.h5
16684/16684 [==============================] - 904s 54ms/step - loss: 0.4843 - accuracy: 0.8371 - val_loss: 0.4810 - val_accuracy: 0.8402
Epoch 3/1000
16682/16684 [============================>.] - ETA: 0s - loss: 0.4569 - accuracy: 0.8444
Epoch 3: val_loss did not improve from 0.48097
16684/16684 [==============================] - 604s 36ms/step - loss: 0.4569 - accuracy: 0.8444 - val_loss: 0.5305 - val_accuracy: 0.8280
Epoch 4/1000
16683/16684 [============================>.] - ETA: 0s -

### adamax

In [ ]:
base_model = tf.keras.applications.ResNet50(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(62, activation="softmax")(avg)
model2 = tf.keras.Model(inputs=base_model.input, outputs=output)
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
early_stopping_BRN50 = EarlyStopping(patience = 10)
data_name="byclass" 
type_name="RN50"
checkpoint_callback_BRN50 = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
model2.compile(optimizer='adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history2=model2.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping_BRN50, checkpoint_callback_BRN50])

Epoch 1/1000
16683/16684 [============================>.] - ETA: 0s - loss: 0.6869 - accuracy: 0.7945
Epoch 1: val_loss improved from inf to 0.52709, saving model to byclass_RN50.h5
16684/16684 [==============================] - 1018s 61ms/step - loss: 0.6869 - accuracy: 0.7945 - val_loss: 0.5271 - val_accuracy: 0.8286
Epoch 2/1000
16683/16684 [============================>.] - ETA: 0s - loss: 0.4949 - accuracy: 0.8365
Epoch 2: val_loss improved from 0.52709 to 0.47900, saving model to byclass_RN50.h5
16684/16684 [==============================] - 1209s 72ms/step - loss: 0.4949 - accuracy: 0.8365 - val_loss: 0.4790 - val_accuracy: 0.8393
Epoch 3/1000
16684/16684 [==============================] - ETA: 0s - loss: 0.4582 - accuracy: 0.8454
Epoch 3: val_loss improved from 0.47900 to 0.45303, saving model to byclass_RN50.h5
16684/16684 [==============================] - 1017s 61ms/step - loss: 0.4582 - accuracy: 0.8454 - val_loss: 0.4530 - val_accuracy: 0.8464
Epoch 4/1000
16684/16684 [===

KeyboardInterrupt: 

### 1epoch당 1000초 이상의 너무 오랜 시간이 걸려 도중에 중지 시켰다

### adamW

In [ ]:
base_model = tf.keras.applications.ResNet50(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(62, activation="softmax")(avg)
model2 = tf.keras.Model(inputs=base_model.input, outputs=output)
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
early_stopping_BRN50 = EarlyStopping(patience = 10)
data_name="byclass" 
type_name="RN50"
#optimizer로 adamW를 사용하는중
optimizer = tfa.optimizers.AdamW(weight_decay=1e-5, learning_rate=0.001,
                                 beta_1=0.9, beta_2=0.999)
checkpoint_callback_BRN50 = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
model2.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history2=model2.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping_BRN50, checkpoint_callback_BRN50])

Epoch 1/1000
16683/16684 [============================>.] - ETA: 0s - loss: 0.5988 - accuracy: 0.8085
Epoch 1: val_loss improved from inf to 0.51102, saving model to byclass_RN50.h5
16684/16684 [==============================] - 1116s 67ms/step - loss: 0.5988 - accuracy: 0.8085 - val_loss: 0.5110 - val_accuracy: 0.8277
Epoch 2/1000
16683/16684 [============================>.] - ETA: 0s - loss: 0.4896 - accuracy: 0.8352
Epoch 2: val_loss improved from 0.51102 to 0.48264, saving model to byclass_RN50.h5
16684/16684 [==============================] - 943s 57ms/step - loss: 0.4896 - accuracy: 0.8352 - val_loss: 0.4826 - val_accuracy: 0.8390
Epoch 3/1000
16684/16684 [==============================] - ETA: 0s - loss: 0.4699 - accuracy: 0.8401
Epoch 3: val_loss did not improve from 0.48264
16684/16684 [==============================] - 1199s 72ms/step - loss: 0.4699 - accuracy: 0.8401 - val_loss: 0.5129 - val_accuracy: 0.8222
Epoch 4/1000
16684/16684 [==============================] - ETA: 0s

# rmsprop

In [ ]:
base_model = tf.keras.applications.ResNet50(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(62, activation="softmax")(avg)
model2 = tf.keras.Model(inputs=base_model.input, outputs=output)
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
early_stopping_BRN50 = EarlyStopping(patience = 10)
data_name="byclass" 
type_name="RN50"
checkpoint_callback_BRN50 = ModelCheckpoint(data_name+"_"+type_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
model2.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
history2=model2.fit(train_set, batch_size = batch_size,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(valid_size / batch_size), 
                    epochs=1000,
                    callbacks=[early_stopping_BRN50, checkpoint_callback_BRN50])

Epoch 1/1000
16684/16684 [==============================] - ETA: 0s - loss: 0.6964 - accuracy: 0.7964
Epoch 1: val_loss improved from inf to 0.67793, saving model to byclass_RN50.h5
16684/16684 [==============================] - 1076s 64ms/step - loss: 0.6964 - accuracy: 0.7964 - val_loss: 0.6779 - val_accuracy: 0.7967
Epoch 2/1000
16684/16684 [==============================] - ETA: 0s - loss: 0.6635 - accuracy: 0.8195
Epoch 2: val_loss did not improve from 0.67793
16684/16684 [==============================] - 1227s 74ms/step - loss: 0.6635 - accuracy: 0.8195 - val_loss: 0.6820 - val_accuracy: 0.8216
Epoch 3/1000
16684/16684 [==============================] - ETA: 0s - loss: 0.6854 - accuracy: 0.8233
Epoch 3: val_loss did not improve from 0.67793
16684/16684 [==============================] - 1052s 63ms/step - loss: 0.6854 - accuracy: 0.8233 - val_loss: 0.7578 - val_accuracy: 0.8110
Epoch 4/1000
16683/16684 [============================>.] - ETA: 0s - loss: 0.7067 - accuracy: 0.8252
E